# CNN for Analysis

In [9]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

In [10]:
# Initializing the CNN
classifier = Sequential()

2024-04-22 15:35:00.022888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# Adding the Convolution Layer
classifier.add(Convolution2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))


In [12]:
# Adding the Pooling Layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))


In [13]:
# Flattening the layer
classifier.add(Flatten())


In [14]:
# Adding the Fully Connected layer
classifier.add(Dense(units=128, activation='relu'))  # Hidden layer
classifier.add(Dense(units=1, activation='sigmoid'))  # Output layer


In [15]:
# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [17]:
import os
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

# Load the metadata CSV file and encode the labels
metadata_path = '../DATA/metadata.csv'
metadata = pd.read_csv(metadata_path)

# Make sure the 'isic_id' column contains the full filename with extension, e.g., 'ISIC_0000000.jpg'
# Assuming your image files are in .jpg format
metadata['isic_id'] = metadata['isic_id'].apply(lambda x: x + '.jpg')

# Encode the 'benign_malignant' labels to numerical values
le = LabelEncoder()
metadata['label'] = le.fit_transform(metadata['benign_malignant'])
# Convert the numerical labels to strings as required by flow_from_dataframe for binary classification
metadata['label'] = metadata['label'].astype(str)

# Define the paths for the training and testing sets
train_set_path = '../DATA/ISIC-images/Train'
test_set_path = '../DATA/ISIC-images/Test'

# Create the ImageDataGenerator for data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Use flow_from_dataframe to load images and labels based on the dataframe
training_set = train_datagen.flow_from_dataframe(
    dataframe=metadata,
    directory=train_set_path,
    x_col='isic_id',  # Column in dataframe that contains the filenames
    y_col='label',  # Column in dataframe that contains the labels
    target_size=(32, 32),
    batch_size=32,
    class_mode='binary'  # Use 'binary' or 'categorical' based on the problem
)

test_set = test_datagen.flow_from_dataframe(
    dataframe=metadata,
    directory=test_set_path,
    x_col='isic_id',
    y_col='label',
    target_size=(32, 32),
    batch_size=32,
    class_mode='binary'
)


# Fit the model to the training set
classifier.fit_generator(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=25,
    validation_data=test_set,
    validation_steps=len(test_set)
)


Found 800 validated image filenames belonging to 2 classes.
Found 200 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 200 invalid image filename(s) in x_col="isic_id". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 800 invalid image filename(s) in x_col="isic_id". These filename(s) will be ignored.
  warnings.warn(
/var/folders/gy/sq_b71qx3931n7lpgb669l0w0000gq/T/ipykernel_21572/2220612322.py:57: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/25
25/25 [==============================] - 67s 3s/step - loss: 0.6356 - accuracy: 0.7550 - val_loss: 0.3897 - val_accuracy: 0.8800
Epoch 2/25
25/25 [==============================] - 70s 3s/step - loss: 0.5162 - accuracy: 0.7763 - val_loss: 0.3811 - val_accuracy: 0.8800
Epoch 3/25
25/25 [==============================] - 66s 3s/step - loss: 0.5066 - accuracy: 0.7862 - val_loss: 0.3951 - val_accuracy: 0.8800
Epoch 4/25
25/25 [==============================] - 75s 3s/step - loss: 0.4882 - accuracy: 0.7950 - val_loss: 0.5015 - val_accuracy: 0.8700
Epoch 5/25
25/25 [==============================] - 66s 3s/step - loss: 0.4994 - accuracy: 0.7937 - val_loss: 0.3928 - val_accuracy: 0.8750
Epoch 6/25
25/25 [==============================] - 66s 3s/step - loss: 0.4713 - accuracy: 0.8025 - val_loss: 0.4231 - val_accuracy: 0.8800
Epoch 7/25
25/25 [==============================] - 67s 3s/step - loss: 0.4805 - accuracy: 0.8075 - val_loss: 0.4320 - val_accuracy: 0.8650
Epoch 8/25
25/25 [==

In [18]:
# download the model
classifier.save('../OUTPUT/classifier_model.h5')